In [1]:
%reload_ext autoreload
%autoreload 2

import os
from pathlib import Path
import multiprocessing

import jax
import pandas as pd
import numpy as np
import numpyro

from hbmep.config import Config
from hbmep.model import Model

PLATFORM = "cpu"
jax.config.update("jax_platforms", PLATFORM)
numpyro.set_platform(PLATFORM)

cpu_count = multiprocessing.cpu_count() - 2
numpyro.set_host_device_count(cpu_count)
numpyro.enable_x64()


#### Load config

In [2]:
root_path = Path(os.getcwd()).parent.parent.parent.absolute()
toml_path = os.path.join(root_path, "configs/simulation.toml")


In [3]:
# Initialize and validate configuration
config = Config(toml_path=toml_path)


2023-07-18 15:29:31,830 - hbmep.config - INFO - Verifying configuration ...
2023-07-18 15:29:31,830 - hbmep.config - INFO - Success!


In [4]:
# Initialize model
model = Model(config=config)

2023-07-18 15:29:31,933 - hbmep.model.core - INFO - Initialized rectified_logistic model


In [5]:
# Load data
df, encoder_dict = model.load()

2023-07-18 15:29:32,287 - hbmep.model.baseline - INFO - Simulating data ...
2023-07-18 15:29:45,841 - hbmep.utils.utils - INFO - func:simulate took: 13.82 sec
2023-07-18 15:29:45,842 - hbmep.dataset.core - INFO - Artefacts will be stored here - /home/vishu/repos/hbmep-paper/reports/simulation/
2023-07-18 15:29:45,842 - hbmep.dataset.core - INFO - Copied config to /home/vishu/repos/hbmep-paper/reports/simulation/
2023-07-18 15:29:45,843 - hbmep.dataset.core - INFO - Processing data ...
2023-07-18 15:29:45,844 - hbmep.utils.utils - INFO - func:load took: 0.00 sec


In [6]:
# Plot dataset
model.plot(df=df, encoder_dict=encoder_dict)

2023-07-18 15:29:45,950 - hbmep.dataset.core - INFO - Plotting dataset ...


2023-07-18 15:29:47,317 - hbmep.dataset.core - INFO - Saved to /home/vishu/repos/hbmep-paper/reports/simulation/dataset.pdf
2023-07-18 15:29:47,318 - hbmep.utils.utils - INFO - func:plot took: 1.37 sec


In [7]:
# Run inference
mcmc, posterior_samples = model.run_inference(df=df)


2023-07-18 15:29:47,419 - hbmep.model.baseline - INFO - Running inference with rectified_logistic ...


  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

2023-07-18 15:36:36,055 - hbmep.utils.utils - INFO - func:run_inference took: 6 min and 48.64 sec


In [8]:
mcmc.print_summary(prob=.95)



                mean       std    median      2.5%     97.5%     n_eff     r_hat
  H[0,0,0]      2.42      0.06      2.42      2.31      2.54  24138.52      1.00
  H[0,0,1]      4.32      3.29      3.26      1.14     10.76  12132.31      1.00
  H[0,1,0]      1.69      2.39      0.82      0.09      6.27  10667.17      1.00
  H[0,1,1]      0.21      0.01      0.21      0.20      0.23  21684.74      1.00
  H[0,2,0]      3.84      4.59      2.31      0.00     13.06   7469.68      1.00
  H[0,2,1]      0.28      0.05      0.27      0.21      0.36   2511.59      1.00
  H[1,0,0]      0.24      0.02      0.24      0.20      0.27  20985.70      1.00
  H[1,0,1]      1.00      0.05      1.00      0.90      1.10  25325.17      1.00
  H[1,1,0]      0.64      0.04      0.64      0.56      0.72  11895.24      1.00
  H[1,1,1]      3.31      0.13      3.31      3.07      3.57  22373.82      1.00
  H[1,2,0]      7.30      4.56      5.99      1.93     16.28  12416.72      1.00
  H[1,2,1]      2.04      2

In [9]:
# Recruitment curves
model.render_recruitment_curves(df=df, encoder_dict=encoder_dict, posterior_samples=posterior_samples)


2023-07-18 15:36:36,541 - hbmep.model.baseline - INFO - Rendering recruitment curves ...
2023-07-18 15:36:53,388 - hbmep.model.baseline - INFO - Saved to /home/vishu/repos/hbmep-paper/reports/simulation/recruitment_curves.pdf
2023-07-18 15:36:53,389 - hbmep.utils.utils - INFO - func:render_recruitment_curves took: 16.85 sec


In [10]:
# Posterior Predictive Check
model.render_predictive_check(df=df, encoder_dict=encoder_dict, posterior_samples=posterior_samples)


2023-07-18 15:36:53,425 - hbmep.model.baseline - INFO - Rendering Posterior Predictive Check ...


2023-07-18 15:37:13,407 - hbmep.model.baseline - INFO - Saved to /home/vishu/repos/hbmep-paper/reports/simulation/posterior_predictive_check.pdf
2023-07-18 15:37:13,410 - hbmep.utils.utils - INFO - func:render_predictive_check took: 19.99 sec
